**Parameter Optimization**

Import Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

FUNCTIONS


In [10]:
#plot

def plot_accuracy(model_history):
    plt.plot(model_history.history['accuracy'], label = 'Training Accuracy')
    plt.plot(model_history.history['val_accuracy'], label = 'Validation Accuracy')
    plt.title("Accuracy of Model")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_loss(model_history):
    plt.plot(model_history.history['loss'], label = 'Training Loss')
    plt.plot(model_history.history['val_loss'], label = 'Validation Loss')
    plt.title("Loss of Model")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.grid(True)
    plt.legend()
    plt.show()

DATA PREPROCESSING

In [5]:
df = pd.read_csv("heart_statlog_cleveland_hungary_final.csv")
df.head(5)

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [6]:
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  918 non-null    int64  
 1   sex                  918 non-null    int64  
 2   chest pain type      918 non-null    int64  
 3   resting bp s         918 non-null    int64  
 4   cholesterol          918 non-null    int64  
 5   fasting blood sugar  918 non-null    int64  
 6   resting ecg          918 non-null    int64  
 7   max heart rate       918 non-null    int64  
 8   exercise angina      918 non-null    int64  
 9   oldpeak              918 non-null    float64
 10  ST slope             918 non-null    int64  
 11  target               918 non-null    int64  
dtypes: float64(1), int64(11)
memory usage: 86.2 KB


In [41]:
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

print(f"First 5 Lines of Input Data: {X[0:6]}")
print(f"\nFirst 5 Lines of Target Data: {y[0:6]}")


First 5 Lines of Input Data: [[ 40.    1.    2.  140.  289.    0.    0.  172.    0.    0.    1. ]
 [ 49.    0.    3.  160.  180.    0.    0.  156.    0.    1.    2. ]
 [ 37.    1.    2.  130.  283.    0.    1.   98.    0.    0.    1. ]
 [ 48.    0.    4.  138.  214.    0.    0.  108.    1.    1.5   2. ]
 [ 54.    1.    3.  150.  195.    0.    0.  122.    0.    0.    1. ]
 [ 39.    1.    3.  120.  339.    0.    0.  170.    0.    0.    1. ]]

First 5 Lines of Target Data: [0 1 0 1 0 0]


In [8]:
#split the data set into training and test data


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.4, random_state = 0)
#X_train and Y_train are now 60% of X/Y_transformed

#X_temp and Y_temp are now 40% of X/Y_transformed
#next line splits this last 40% in two parts, one for validation (20% of X/Y_transformed) and one for testing (20% of X/Y_transformed)

X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 0)

In [9]:
#scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [11]:
#Initial regularization Values
regularization_values = [0.1, 0.01, 0.001, 0.0001]

#regularization values refined
m1_reg_refined = [0.001,0.0019,0.0028,0.0037,0.0046,0.0055,0.0064,0.0073,0.0082,0.0091]
m2_reg_refined = [0.00018, 0.00026, 0.00034, 0.00042, 0.0005, 0.00059, 0.00067, 0.00075, 0.00083, 0.00091]
m3_reg_refined = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]
m4_reg_refined = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

#Final Refined Regularization Values
m1_reg = [0.001, 0.0064]
m2_reg = [0.00026, 0.00067]
m3_reg = [0.00072, 0.00083]
m4_reg = [0.012, 0.013]

In [31]:
#Refined Regularization Values with Learning Rate
m1_reg_lr = [0.001]
m2_reg_lr = [0.00018, 0.00026, 0.00067]
m3_reg_lr = [0.00072, 0.00083, 0.0014, 0.0015]
m4_reg_lr = [0.012, 0.013]

#learning rate values 
learning_rates = [0.1, 0.01, 0.001, 0.0001]

In [33]:
#Learning rate Refined
'''
MODEL 1 INFO:
Model 1 Regularization Parameter: 0.001
Model 1 learning rate refined: 0.01 +-, m1_lr_refined
'''
m1_lr_refined = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

'''
MODEL 2 INFO: 
Model 2 Regularization Parameters Refined: 1 - 0.00018, 2 - 0.00026, 3 - 0.00067
Model 2 Regulaization Parameter 1 Learning Rate refined: 0.001 +-, m2_lr_refined_1
Model 2 Regularization Paramter 2 learning Rate Refined: 0.01 +-, m2_lr_refined_2
Model 2 Regulaization Parameter 3 Learning Rate Refined: 0.01 +-, m2_lr_refined_2
'''
m2_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]
m2_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

'''
MODEL 3 INFO:
Model 3 Regularization Parameters Refined: 1 - 0.00072, 2 - 0.00083, 3 - 0.0014, 4 - 0.0015
Model 3 Regulaization Parameter 1 Learning Rate refined: 0.001 +-, m5_lr_refined_1
Model 3 Regulaization Parameter 2 Learning Rate refined: 0.01 +-, m5_lr_refined_2
Model 3 Regulaization Parameter 3 Learning Rate refined: 0.01 +-, m5_lr_refined_2
Model 3 Regulaization Parameter 4 Learning Rate refined: 0.01 +-, m5_lr_refined_2
'''
m3_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]
m3_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

'''
MODEL 4 INFO:
Model 4 Regularization Parameters Refined: 1 - 0.012, 2 - 0.013
Model 4 Regulaization Parameter 1 Learning Rate refined: 0.01 to 0.001, m7_lr_refined_1
Model 4 Regulaization Parameter 2 Learning Rate refined: 0.001 to 0.0001, m7_lr_refined_2
'''
m4_lr_refined_1 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.010]
m4_lr_refined_2 = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]

**MODEL 1**

In [12]:
'''
Use the this and the following four code blocks to do an initial test on Model 1's structure
'''

model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', name='L1'),
    tf.keras.layers.Dense(16, activation='relu', name='L2'),
    tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
])

model_1.compile(optimizer = tf.keras.optimizers.Adam(),
            loss = tf.keras.losses.BinaryCrossentropy(), 
            metrics = ['accuracy'])

model_1_history = model_1.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

In [ ]:
plot_accuracy(model_1_history)
plot_loss(model_1_history)

In [ ]:
model_1.evaluate(X_test, y_test)

In [ ]:
y_predicted_m1 = model_1.predict(X_test)
y_predicted_m1 = y_predicted_m1.flatten()
y_predicted_m1 = np.where(y_predicted_m1 > 0.5, 1, 0)

print(type(y_test))
print(type(y_predicted_m1))

cm = confusion_matrix(y_test, y_predicted_m1)
print(cm)

sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [19]:
'''
Use the following for loop to train and evaluate Model 1 with the generalized set of regularization parameters
'''

# for i in regularization_values:
#     print(f"REGULARIZATION PARAMETER: {i}")
    
#     model_1_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(16, activation='relu', name='L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(16, activation='relu', name='L2', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#     ])

#     model_1_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_1_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     plot_accuracy(history)
#     plot_loss(history)

#     print("\nModel Prediction on Test set")
#     results= model_1_reg.evaluate(X_test, y_test)

#     y_predicted_m1_reg = model_1_reg.predict(X_test)
#     y_predicted_m1_reg = y_predicted_m1_reg.flatten()
#     y_predicted_m1_reg = np.where(y_predicted_m1_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m1_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m1_reg))    

In [ ]:
'''
Use the following for loop to train and evaluate Model 1 with its refined set of regularization parameters
'''

# for i in m1_reg_refined:
#     print(f"REGULARIZATION PARAMETER: {i}")
    
#     model_1_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(16, activation='relu', name='L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(16, activation='relu', name='L2', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#     ])

#     model_1_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_1_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     plot_accuracy(history)
#     plot_loss(history)

#     print("\nModel Prediction on Test set")
#     results= model_1_reg.evaluate(X_test, y_test)

#     y_predicted_m1_reg = model_1_reg.predict(X_test)
#     y_predicted_m1_reg = y_predicted_m1_reg.flatten()
#     y_predicted_m1_reg = np.where(y_predicted_m1_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m1_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m1_reg)) 

In [30]:
'''
Use the following for loop to train and evaluate Model 1 with its refined regularization values,
and its generalized set of learning rates.
'''

# for reg in m1_reg:
#     for lr in learning_rates:
#             print(f"REGULARIZATION PARAMETER: {reg}")
#             print(f"LEARNING RATE = {lr}")
            
#             model_1_reg_lr = tf.keras.Sequential([
#                 tf.keras.layers.Dense(16, activation='relu', name='L1', kernel_regularizer = l2(reg)),
#                 tf.keras.layers.Dense(16, activation='relu', name='L2', kernel_regularizer = l2(reg)),
#                 tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#             ])
        
#             model_1_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                     loss = tf.keras.losses.BinaryCrossentropy(), 
#                     metrics = ['accuracy'])
        
#             history = model_1_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
        
#             plot_accuracy(history)
#             plot_loss(history)
        
#             print("\nModel Prediction on Test set")
#             results= model_1_reg_lr.evaluate(X_test, y_test)
        
#             y_predicted_m1 = model_1_reg_lr.predict(X_test)
#             y_predicted_m1 = y_predicted_m1.flatten()
#             y_predicted_m1 = np.where(y_predicted_m1 > 0.5, 1, 0)
        
#             print(f"\nConfusion Matrix")
#             cm = confusion_matrix(y_test, y_predicted_m1)
#             print(cm)
        
#             print(classification_report(y_test, y_predicted_m1))

In [1]:
'''
Use the following for loop to train and eevaluate Model 1 with its final regularization values, 
and refined learning rates.
'''

# for reg in m1_reg_lr:
#     for lr in m1_lr_refined:
#         print(f"REGULARIZATION PARAMETER: {reg}")
#         print(f"LEARNING RATE = {lr}")
            
#         model_1_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(16, activation='relu', name='L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(16, activation='relu', name='L2', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#         ])
        
#         model_1_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
        
#         history = model_1_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
        
#         plot_accuracy(history)
#         plot_loss(history)
        
#         print("\nModel Prediction on Test set")
#         results= model_1_reg_lr.evaluate(X_test, y_test)
        
#         y_predicted_m1 = model_1_reg_lr.predict(X_test)
#         y_predicted_m1 = y_predicted_m1.flatten()
#         y_predicted_m1 = np.where(y_predicted_m1 > 0.5, 1, 0)
        
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m1)
#         print(cm)
        
#         print(classification_report(y_test, y_predicted_m1))

In [ ]:
'''
Test Final Configuration of Model 1
'''

# m1_reg_final = 0.001
# m1_learning_rate_final = 0.011

# print(f"REGULARIZATION PARAMETER: {m1_reg_final}")
# print(f"LEARNING RATE = {m1_learning_rate_final}")
                    
# model_1 = tf.keras.Sequential([
#     tf.keras.layers.Dense(16, activation='relu', name='L1', kernel_regularizer = l2(m1_reg_final)),
#     tf.keras.layers.Dense(16, activation='relu', name='L2', kernel_regularizer = l2(m1_reg_final)),
#     tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
# ])
                
# model_1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = m1_learning_rate_final),
#                        loss = tf.keras.losses.BinaryCrossentropy(),  
#                        metrics = ['accuracy'])
                
# history = model_1.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
                
# plot_accuracy(history)
# plot_loss(history)
                
# print("\nModel Prediction on Test set")
# results= model_1.evaluate(X_test, y_test)
                
# y_predicted_m1 = model_1.predict(X_test)
# y_predicted_m1 = y_predicted_m1.flatten()
# y_predicted_m1 = np.where(y_predicted_m1 > 0.5, 1, 0)
            
# print(f"\nConfusion Matrix")
# cm = confusion_matrix(y_test, y_predicted_m1)
# print(cm)

# print(classification_report(y_test, y_predicted_m1))


**MODEL 2**

In [29]:
'''
Use the this and the following four code blocks to do an initial test on Model 2's structure
'''

model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(25, activation = 'relu', name = 'L1'),
    tf.keras.layers.Dense(15, activation = 'relu', name = 'L2'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
])

model_2.compile(optimizer = tf.keras.optimizers.Adam(),
            loss = tf.keras.losses.BinaryCrossentropy(), 
            metrics = ['accuracy'])

model_2_history = model_2.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

In [ ]:
plot_accuracy(model_2_history)
plot_loss(model_2_history)

In [ ]:
y_predicted_m2 = model_2.predict(X_test)
y_predicted_m2 = y_predicted_m2.flatten()
y_predicted_m2 = np.where(y_predicted_m2 > 0.5, 1, 0)

print(type(y_test))
print(type(y_predicted_m2))

cm = confusion_matrix(y_test, y_predicted_m2)
print(cm)

sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
'''
Use the following for loop to train and evaluate Model 2 with the generalized set of regularization parameters
'''

# for i in regularization_values:
#     print(f"REGULARIZATION PARAMETER {i}")
    
#     model_2_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(25, activation = 'relu', name = 'L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(15, activation = 'relu', name = 'L2', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#     ])

#     model_2_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_2_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     print("\nModel Prediction on Test set")
#     results= model_2_reg.evaluate(X_test, y_test)

#     plot_accuracy(history)
#     plot_loss(history)

#     y_predicted_m2_reg = model_2_reg.predict(X_test)
#     y_predicted_m2_reg = y_predicted_m2_reg.flatten()
#     y_predicted_m2_reg = np.where(y_predicted_m2_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m2_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m2_reg))

In [29]:
'''
Use the following for loop to train and evaluate Model 2 with its refined set of regularization parameters
'''

# for i in m2_reg_refined:
#     print(f"REGULARIZATION PARAMETER {i}")
    
#     model_2_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(25, activation = 'relu', name = 'L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(15, activation = 'relu', name = 'L2', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#     ])

#     model_2_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_2_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     print("\nModel Prediction on Test set")
#     results= model_2_reg.evaluate(X_test, y_test)

#     plot_accuracy(history)
#     plot_loss(history)

#     y_predicted_m2_reg = model_2_reg.predict(X_test)
#     y_predicted_m2_reg = y_predicted_m2_reg.flatten()
#     y_predicted_m2_reg = np.where(y_predicted_m2_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m2_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m2_reg))

In [2]:
'''
Use the following for loop to train and evaluate Model 2 with its refined regularization values,
and its generalized set of learning rates.
'''

# for reg in m2_reg:
#     for lr in learning_rates:
#         print(f"REGULARIZATION PARAMETER {reg}")
#         print(f"LEARNING RATE = {lr}")
        
#         model_2_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(25, activation = 'relu', name = 'L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(15, activation = 'relu', name = 'L2', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#         ])
    
#         model_2_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
    
#         history = model_2_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
    
#         print("\nModel Prediction on Test set")
#         results= model_2_reg_lr.evaluate(X_test, y_test)
    
#         plot_accuracy(history)
#         plot_loss(history)
    
#         y_predicted_m2 = model_2_reg_lr.predict(X_test)
#         y_predicted_m2 = y_predicted_m2.flatten()
#         y_predicted_m2 = np.where(y_predicted_m2 > 0.5, 1, 0)
    
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m2)
#         print(cm)
    
#         print(classification_report(y_test, y_predicted_m2))
    

'\nUse the following for loop to train and evaluate Model 1 with its refined regularization values,\nand its generalized set of learning rates.\n'

In [3]:
'''
Refined Regularization Parameters: m2_reg_lr = [0.00018, 0.00026, 0.00067]

These are the refined Learning rates for each Regularization parameter

0.00018 -> m2_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]
0.00026, 0.00067 -> m2_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

Use the following for loop to train and eevaluate Model 2 with its final regularization values, 
and refined learning rates.
'''

# for reg in m2_reg_lr:           
    
#     learning_rate_for_reg_param = []
    
#     if reg == m2_reg_lr[0]:
#         learning_rate_for_reg_param = m2_lr_refined_1
#     else:
#         learning_rate_for_reg_param = m2_lr_refined_2
        
#     for lr in learning_rate_for_reg_param:
#         print(f"REGULARIZATION PARAMETER {reg}")
#         print(f"LEARNING RATE = {lr}")
            
#         model_2_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(25, activation = 'relu', name = 'L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(15, activation = 'relu', name = 'L2', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#             ])
        
#         model_2_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
        
#         history = model_2_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
        
#         print("\nModel Prediction on Test set")
#         results= model_2_reg_lr.evaluate(X_test, y_test)
        
#         plot_accuracy(history)
#         plot_loss(history)
        
#         y_predicted_m2 = model_2_reg_lr.predict(X_test)
#         y_predicted_m2 = y_predicted_m2.flatten()
#         y_predicted_m2 = np.where(y_predicted_m2 > 0.5, 1, 0)
        
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m2)
#         print(cm)
        
#         print(classification_report(y_test, y_predicted_m2))

'\nUse the following for loop to train and eevaluate Model 2 with its final regularization values, \nand refined learning rates.\n'

**MODEL 3**

In [14]:
'''
Use the this and the following four code blocks to do an initial test on Model 3's structure
'''

model_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu', name = 'L1'),
    tf.keras.layers.Dense(32, activation = 'relu', name = 'L2'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
])

model_3.compile(optimizer = tf.keras.optimizers.Adam(),
            loss = tf.keras.losses.BinaryCrossentropy(), 
            metrics = ['accuracy'])

model_3_history = model_3.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

In [ ]:
plot_accuracy(model_3_history)
plot_loss(model_3_history)

In [ ]:
model_3.evaluate(X_test, y_test)

In [ ]:
y_predicted_m3 = model_3.predict(X_test)
y_predicted_m3 = y_predicted_m3.flatten()
y_predicted_m3 = np.where(y_predicted_m3 > 0.5, 1, 0)

print(type(y_test))
print(type(y_predicted_m3))

cm = confusion_matrix(y_test, y_predicted_m3)
print(cm)

sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
'''
Use the following for loop to train and evaluate Model 3 with the generalized set of regularization parameters
'''

# for i in regularization_values:

#     print(f"REGULARIZATION PARAMETER {i}")

#     model_3_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation = 'relu', name = 'L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(32, activation = 'relu', name = 'L2', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#     ])

#     model_3_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_3_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     print("\nModel Prediction on Test set")
#     results= model_5_reg.evaluate(X_test, y_test)

#     plot_accuracy(history)
#     plot_loss(history)

#     y_predicted_m3_reg = model_3_reg.predict(X_test)
#     y_predicted_m3_reg = y_predicted_m3_reg.flatten()
#     y_predicted_m3_reg = np.where(y_predicted_m3_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m3_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m3_reg))

In [24]:
'''
Use the following for loop to train and evaluate Model 3 with its refined set of regularization parameters
'''

# for i in m3_reg_refined:

#     print(f"REGULARIZATION PARAMETER {i}")

#     model_3_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation = 'relu', name = 'L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(32, activation = 'relu', name = 'L2', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#     ])

#     model_3_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_3_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     print("\nModel Prediction on Test set")
#     results= model_3_reg.evaluate(X_test, y_test)

#     plot_accuracy(history)
#     plot_loss(history)

#     y_predicted_m3_reg = model_3_reg.predict(X_test)
#     y_predicted_m3_reg = y_predicted_m3_reg.flatten()
#     y_predicted_m3_reg = np.where(y_predicted_m3_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m3_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m3_reg))

In [4]:
'''
Use the following for loop to train and evaluate Model 3 with its refined regularization values,
and its generalized set of learning rates.
'''

# for reg in m5_reg:
#     for lr in learning_rates:
#         print(f"REGULARIZATION PARAMETER {reg}")
#         print(f"LEARNING RATE = {lr}")
    
#         model_5_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(64, activation = 'relu', name = 'L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(32, activation = 'relu', name = 'L2', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#         ])
    
#         model_5_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
    
#         history = model_5_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
    
#         print("\nModel Prediction on Test set")
#         results= model_5_reg_lr.evaluate(X_test, y_test)
    
#         plot_accuracy(history)
#         plot_loss(history)
    
#         y_predicted_m5 = model_5_reg_lr.predict(X_test)
#         y_predicted_m5 = y_predicted_m5.flatten()
#         y_predicted_m5 = np.where(y_predicted_m5 > 0.5, 1, 0)
    
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m5)
#         print(cm)
    
#         print(classification_report(y_test, y_predicted_m5))

'\nUse the following for loop to train and evaluate Model 3 with its refined regularization values,\nand its generalized set of learning rates.\n'

In [5]:
'''
Refined Regularization Parameters: m3_reg_lr = [0.00072, 0.00083, 0.0014, 0.0015]

These are the refined Learning rates for each Regularization parameter
m3_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]
m3_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

0.00072 -> m3_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]
0.00083, 0.0014, 0.0015 -> m3_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]

Use the following for loop to train and eevaluate Model 3 with its final regularization values, 
and refined learning rates.
'''

# for reg in m3_reg_lr:

#     learning_rate_for_reg_param = []

#     if reg == m3_reg_lr[0]:
#         learning_rate_for_reg_param = m3_lr_refined_1
#     else: 
#         learning_rate_for_reg_param = m3_lr_refined_2
        
#     for lr in learning_rate_for_reg_param:
#         print(f"REGULARIZATION PARAMETER {reg}")
#         print(f"LEARNING RATE = {lr}")
    
#         model_3_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(64, activation = 'relu', name = 'L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(32, activation = 'relu', name = 'L2', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'OL')
#         ])
    
#         model_3_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
    
#         history = model_3_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
    
#         print("\nModel Prediction on Test set")
#         results= model_3_reg_lr.evaluate(X_test, y_test)
    
#         plot_accuracy(history)
#         plot_loss(history)
    
#         y_predicted_m3 = model_3_reg_lr.predict(X_test)
#         y_predicted_m3 = y_predicted_m3.flatten()
#         y_predicted_m3 = np.where(y_predicted_m3 > 0.5, 1, 0)
    
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m3)
#         print(cm)
    
#         print(classification_report(y_test, y_predicted_m3))

'\nRefined Regularization Parameters: m5_reg_lr = [0.00072, 0.00083, 0.0014, 0.0015]\n\nThese are the refined Learning rates for each Regularization parameter\nm5_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]\nm5_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]\n\n0.00072 -> m5_lr_refined_1 = [0.0005, 0.00061, 0.00072, 0.00083, 0.00094, 0.001, 0.0011, 0.0012, 0.0013, 0.0014, 0.0015]\n0.00083, 0.0014, 0.0015 -> m5_lr_refined_2 = [0.0054, 0.0063, 0.0072, 0.0081, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015]\n\nUse the following for loop to train and eevaluate Model 2 with its final regularization values, \nand refined learning rates.\n'

**MODEL 4**

In [ ]:
'''
Use the this and the following four code blocks to do an initial test on Model 4's structure
'''


model_4 = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', name='L1'),
    tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
])

model_4.compile(optimizer = tf.keras.optimizers.Adam(),
            loss = tf.keras.losses.BinaryCrossentropy(), 
            metrics = ['accuracy'])

model_4_history = model_4.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

In [ ]:
plot_accuracy(model_4_history)
plot_loss(model_4_history)

In [ ]:
model_4.evaluate(X_test, y_test)

In [ ]:
y_predicted_m4 = model_4.predict(X_test)
y_predicted_m4 = y_predicted_m4.flatten()
y_predicted_m4 = np.where(y_predicted_m4 > 0.5, 1, 0)

print(type(y_test))
print(type(y_predicted_m4))

cm = confusion_matrix(y_test, y_predicted_m4)
print(cm)

sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
'''
Use the following for loop to train and evaluate Model 3 with the generalized set of regularization parameters
'''
# for i in regularization _values:

#     print(f"REGULARIZATION PARAMETER {i}")

#     model_4_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', name='L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#     ])


#     model_4_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_4_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     print("\nModel Prediction on Test set")
#     results= model_4_reg.evaluate(X_test, y_test)

#     plot_accuracy(history)
#     plot_loss(history)

#     y_predicted_m4_reg = model_4_reg.predict(X_test)
#     y_predicted_m4_reg = y_predicted_m4_reg.flatten()
#     y_predicted_m4_reg = np.where(y_predicted_m4_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m4_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m4_reg))

In [27]:
'''
Use the following for loop to train and evaluate Model 4 with its refined set of regularization parameters
'''

# for i in m4_reg_refined:

#     print(f"REGULARIZATION PARAMETER {i}")

#     model_4_reg = tf.keras.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', name='L1', kernel_regularizer = l2(i)),
#         tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#     ])


#     model_4_reg.compile(optimizer = tf.keras.optimizers.Adam(),
#             loss = tf.keras.losses.BinaryCrossentropy(), 
#             metrics = ['accuracy'])

#     history = model_4_reg.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))

#     print("\nModel Prediction on Test set")
#     results= model_4_reg.evaluate(X_test, y_test)

#     plot_accuracy(history)
#     plot_loss(history)

#     y_predicted_m4_reg = model_4_reg.predict(X_test)
#     y_predicted_m4_reg = y_predicted_m4_reg.flatten()
#     y_predicted_m4_reg = np.where(y_predicted_m4_reg > 0.5, 1, 0)

#     print(f"\nConfusion Matrix")
#     cm = confusion_matrix(y_test, y_predicted_m4_reg)
#     print(cm)

#     print(classification_report(y_test, y_predicted_m4_reg))

In [6]:
'''
Use the following for loop to train and evaluate Model 3 with its refined regularization values,
and its generalized set of learning rates.
'''
# for reg in m7_reg:
#     for lr in learning_rates:
#         print(f"REGULARIZATION PARAMETER {reg}")
#         print(f"LEARNING RATE = {lr}")
    
#         model_7_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(256, activation='relu', name='L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#         ])
    
    
#         model_7_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
    
#         history = model_7_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
    
#         print("\nModel Prediction on Test set")
#         results= model_7_reg_lr.evaluate(X_test, y_test)
    
#         plot_accuracy(history)
#         plot_loss(history)
    
#         y_predicted_m7 = model_7_reg_lr.predict(X_test)
#         y_predicted_m7 = y_predicted_m7.flatten()
#         y_predicted_m7 = np.where(y_predicted_m7 > 0.5, 1, 0)
    
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m7)
#         print(cm)
    
#         print(classification_report(y_test, y_predicted_m7))

'\nUse the following for loop to train and evaluate Model 3 with its refined regularization values,\nand its generalized set of learning rates.\n'

In [7]:
'''
Refined Regularization Parameters: m4_reg_lr = [0.012, 0.013]

These are the refined Learning rates for each Regularization parameter
m4_lr_refined_1 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.010]
m4_lr_refined_2 = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]

0.012 -> m4_lr_refined_1 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.010]
0.013 -> m4_lr_refined_2 = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]

Use the following for loop to train and eevaluate Model 3 with its final regularization values, 
and refined learning rates.
'''



# for reg in m7_reg_lr:

#     learning_rate_for_reg_param = []

#     if reg == m7_reg_lr[0]:
#         learning_rate_for_reg_param = m7_lr_refined_1
#     else:
#         learning_rate_for_reg_param = m7_lr_refined_2
        
#     for lr in learning_rate_for_reg_param:
#         print(f"REGULARIZATION PARAMETER {reg}")
#         print(f"LEARNING RATE = {lr}")
    
#         model_7_reg_lr = tf.keras.Sequential([
#             tf.keras.layers.Dense(256, activation='relu', name='L1', kernel_regularizer = l2(reg)),
#             tf.keras.layers.Dense(1, activation='sigmoid', name='OL')
#         ])
    
#         model_7_reg_lr.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
#                 loss = tf.keras.losses.BinaryCrossentropy(), 
#                 metrics = ['accuracy'])
    
#         history = model_7_reg_lr.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_valid, y_valid))
    
#         print("\nModel Prediction on Test set")
#         results= model_7_reg_lr.evaluate(X_test, y_test)
    
#         plot_accuracy(history)
#         plot_loss(history)
    
#         y_predicted_m7 = model_7_reg_lr.predict(X_test)
#         y_predicted_m7 = y_predicted_m7.flatten()
#         y_predicted_m7 = np.where(y_predicted_m7 > 0.5, 1, 0)
    
#         print(f"\nConfusion Matrix")
#         cm = confusion_matrix(y_test, y_predicted_m7)
#         print(cm)
    
#         print(classification_report(y_test, y_predicted_m7))

'\nRefined Regularization Parameters: m4_reg_lr = [0.012, 0.013]\n\nThese are the refined Learning rates for each Regularization parameter\nm4_lr_refined_1 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.010]\nm4_lr_refined_2 = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]\n\n0.012 -> m4_lr_refined_1 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.010]\n0.013 -> m4_lr_refined_2 = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]\n\nUse the following for loop to train and eevaluate Model 3 with its final regularization values, \nand refined learning rates.\n'